# Filtros de tendencia de más largo plazo: Posición Stan Weinstein Inversores (MM30w)+Posición Stan Weinstein Operadores (MM10w)

Informa tendencia estructural según posición, según teorías de Stan Weinstein (autor del bestseller "Secretos para ganar dinero en los mercados alcistas y bajistas". Permite observar estados y también cambios de estado.

# INDICE DE FUNCIONES:

-filterStanW(activos, report = False):
    Esta función analiza la posición del precio respecto de las medias móviles exponenciales de 10 y 30 semanas para el listado de activos 
    solicitado, siguiendo la teoría de Stan Weinstein. El precio respecto a la MM de 10 semanas es para los "Operadores" o traders de más
    corto plazo, mientras que la MM de 30 semanas es para los "Inversores" o traders de más largo plazo.
    Evalúa dos métricas: 
     - Distancia: La posición del precio respecto de la la MM de 50 ruedas respecto de la MM (10 o 30 week) (si está por encima se considera 
     en posición "Alcista", y si está por debajo se considera posición "Bajista".
    - Fortaleza: la evolución de esa posición definiéndose que si en las últimas 3 semanas esa distancia se amplía al alza,
    la fortaleza es "Creciente" para una tendencia alcista o "Decreciente" si esa distancia se acorta. Viceversa para una
    tendencia Bajista. Nota: debe haber un crecimiento o decrecimiento consecutivo en las 3 semanas para catalogarse como
    "Creciente" o "Decreciente", sino será "No determinada".
    
   Devuelve una tabla con todas las métricas, otra tabla estilada con colores y dos listados con los activos alcistas (operadores e
    inversores).

-getStanSignals(activos, tipo):
    Esta función devuelve los cruces vigentes del precio respecto de las medias exponenciales de 10 o 30 semanas del listado de activos solicitado, así como el sentido del cruce, la fecha de la señal, el precio de la señal, el rendimiento acumulado a la fecha
y el rendimiento acumulado, todo ordenado en forma descendente por cantidad de días transcurridos desde la señal.

In [1]:
# FUNCIONES NECESARIAS PARA OBTENER DATA FINANCIERA:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

def getDataYf(ticker, tipo, interval, data_from = None, data_to = None, period = None):
    """
    Es una función para descargar market data de Yahoo Finance con la librería yfinance.
    
    ## Inputs:
        >ticker: el nombre del ticker.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    if tipo == "no end":
        data = yf.download(ticker, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(ticker, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(ticker, interval = interval, period = period, progress = False, auto_adjust = True)
    return data



def getDataYfMulti(activos, tipo, interval, data_from = None, data_to = None, period = None, swap = True):
    """
    Función para hacer batch requests (varios tickers a la vez), que será la fx que más voy a utilizar para market data.
    
    ## Inputs:
        >tickers: es una lista con los tickers de los cuales se va a obtener market data.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        >swap : si es True, divide el df en tickers y cada uno tiene su OHLC. Si es False, tenemos cada columna OHLC y dentro todos los tickers.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    lideres_arg = ["ALUA.BA", "BBAR.BA", "BMA.BA", "BYMA.BA", "CEPU.BA", "COME.BA", "CRES.BA", "CVH.BA", "EDN.BA", 
                   "GGAL.BA", "LOMA.BA", "MIRG.BA", "PAMP.BA", "SUPV.BA", "TECO2.BA", "TGNO4.BA", "TGSU2.BA", "TRAN.BA", 
                   "TXAR.BA", "VALO.BA", "YPFD.BA"]

    general_arg = ["AGRO.BA", "AUSO.BA", "BHIP.BA", "BOLT.BA", "BPAT.BA", "CADO.BA", "CAPX.BA", "CARC.BA", "CECO2.BA", 
                   "CELU.BA", "CGPA2.BA", "CTIO.BA", "DGCU2.BA", "FERR.BA", "FIPL.BA", "GAMI.BA", "GCDI.BA", "GCLA.BA", 
                   "HARG.BA", "HAVA.BA", "INVJ.BA", "IRSA.BA", "LEDE.BA", "LONG.BA", "METR.BA", "MOLA.BA", "MOLI.BA", 
                   "MORI.BA", "OEST.BA", "PATA.BA", "RICH.BA", "RIGO.BA", "SAMI.BA", "SEMI.BA"]

    cedears = ["AAL", "AAPL", "ABBV", "ABEV", "ABNB", "ABT", "ADBE", "ADGO", "ADI", "ADP", "AEM", "AIG", "AMAT", "AMD", 
               "AMGN", "AMZN", "AOCA", "ARCO", "ARKK", "ASR", "AUY", "AVGO", "AXP", "AZN", "BA", "BA.C", "BABA", "BB", 
               "BBD", "BBV", "BCS", "BHP", "BIDU", "BIIB", "BIOX", "BITF", "BK", "BMY", "BNG", "BP", "BRFS", "BRKB", "BSBR", 
               "C", "CAAP", "CAH", "CAR", "CAT", "CBRD", "CDE", "CL", "COIN", "COST", "CRM", "CS", "CSCO", "CVX", "CX", "DD", 
               "DE", "DESP", "DIA", "DISN", "DOCU", "DOW", "E", "EA", "EBAY", "EEM", "EFX", "ERIC", "ERJ", "ETSY", "EWZ", "F", 
               "FCX", "FDX", "FMX", "FSLR", "GE", "GFI", "GGB", "GILD", "GLOB", "GLW", "GM", "GOLD", "GOOGL", "GPRK", "GRMN", 
               "GS", "HAL", "HD", "HL", "HMC", "HMY", "HOG", "HON", "HPQ", "HSBC", "HSY", "HUT", "HWM", "IBM", "IFF", "INTC", 
               "ITUB", "IWM", "JD", "JMIA", "JNJ", "JPM", "KMB", "KO", "KOFM", "LLY", "LMT", "LRCX", "LVS", "LYG", "MA", "MCD", 
               "MDT", "MELI", "META", "MMM", "MO", "MOS", "MRK", "MSFT", "MSI", "MSTR", "MU", "NEM", "NFLX", "NGG", "NIO", "NKE", 
               "NOKA", "NTCO", "NTES", "NUE", "NVDA", "NVS", "ORAN", "ORCL", "OXY", "PAAS", "PAC", "PANW", "PBI", "PBR", "PCAR", 
               "PEP", "PFE", "PG", "PHG", "PKS", "PSX", "PYPL", "QCOM", "QQQ", "RBLX", "RIO", "RTX", "SAN", "SAP", "SATL", "SBUX", 
               "SCCO", "SE", "SHEL", "SHOP", "SI", "SID", "SLB", "SNAP", "SNOW", "SONY", "SPGI", "SPOT", "SPY", "SQ", "SYY", "T", 
               "TEFO", "TEN", "TGT", "TM", "TMO", "TRIP", "TRVV", "TSLA", "TSM", "TTE", "TV", "TWLO", "TXN", "TXR", "UAL", "UBER", 
               "UGP", "UL", "UNH", "UNP", "UPST", "USB", "V", "VALE", "VIST", "VIV", "VOD", "VZ", "WBA", "WFC", "WMT", "X", "XLE", 
               "XLF", "XOM", "XP", "YY", "ZM"]

    adrs = ["BBAR", "BMA", "CEPU", "CRESY", "EDN", "GGAL", "IRS", "LOMA", "PAM", "SUPV", "TEO", "TGS", "TS", "TX", "YPF"]

    sectors = ["XLC", "XLP", "XLY", "XLF", "XLV", "XLI", "XLRE", "XLU", "XBI", "XLB", "XLK", "XLE"]
    
    precarga = ["lideres", "general", "cedears", "adrs", "sectores"]
    precarga_dict = {"lideres" : lideres_arg, "general" : general_arg, "cedears" : cedears, "adrs" : adrs, "sectores" : sectors}
    
    if activos in precarga:
        activos = precarga_dict[activos]
    
    if tipo == "no end":
        data = yf.download(activos, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(activos, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(activos, interval = interval, period= period, progress = False, auto_adjust = True)
    
    if swap:
        #data = data.swaplevel(i = 0, j = 1, axis = 1)
        # Algoritmo para procesar el MultipleTicker download de yfinance
        dicto = {}
        low = data["Low"]
        high = data["High"]
        close = data["Close"]
        open = data["Open"]
        volume = data["Volume"]

        tickers = list(data["Close"].columns)

        for ticker in tickers:
            dicto[ticker] = {
                "Open" : open[ticker],
                "High" : high[ticker],
                "Low" : low[ticker],
                "Close" : close[ticker],
                "Volume" : volume[ticker]
            }

            dicto[ticker] = pd.DataFrame(dicto[ticker])
        return dicto
    return data

In [13]:
# FUNCIONALIDADES PRINCIPALES DEL MÓDULO:

def filterStanW(activos, report = False):
    """
    Esta función analiza la posición del precio respecto de las medias móviles exponenciales de 10 y 30 semanas para el listado de activos 
    solicitado, siguiendo la teoría de Stan Weinstein. El precio respecto a la MM de 10 semanas es para los "Operadores" o traders de más
    corto plazo, mientras que la MM de 30 semanas es para los "Inversores" o traders de más largo plazo.
    Evalúa dos métricas: 
     - Distancia: La posición del precio respecto de la la MM de 50 ruedas respecto de la MM (10 o 30 week) (si está por encima se considera 
     en posición "Alcista", y si está por debajo se considera posición "Bajista".
    - Fortaleza: la evolución de esa posición definiéndose que si en las últimas 3 semanas esa distancia se amplía al alza,
    la fortaleza es "Creciente" para una tendencia alcista o "Decreciente" si esa distancia se acorta. Viceversa para una
    tendencia Bajista. Nota: debe haber un crecimiento o decrecimiento consecutivo en las 3 semanas para catalogarse como
    "Creciente" o "Decreciente", sino será "No determinada".
    
    Devuelve una tabla con todas las métricas, otra tabla estilada con colores y dos listados con los activos alcistas (operadores e
    inversores).
                       
    # Inputs:
        - activos: es una lista de activos a analizar. Ej.: adrs = ["GGAL", "BBAR", "LOMA", "CEPU"]. También hay sets de activos preconfigurados en la función:
            - "lideres": si se desea cargar todo el panel lider de Argentina;
            - "general": si se desea cargar todo el panel general de Argentina;
            - "adrs": si se desea cargar todo el listado de ADRs argentinos (en usd);
            - "sectors": si se desea cargar el listado de sectores de la economía (ETFs representativos de USA);
            - "cedears": si se desea cargar todo el listado de certificados de depósito de activos del exterior que cotizan en Argentina (puede demorar bastante).
        - excel: si se pasa "True" como argumento, genera un archivo excel en el directorio del script con la tabla estilada.
        
    # Outputs:
        - tabla_final_activos_en_filas: es un DF resumen con los activos en las filas y las variables en las columnas.
        - tabla_final_activos_en_columnas: es un DF resumen con los activos en las columnas y las variables en las filas.
        - estilado: tabla_final_activos_en_filas estilada con colores rojos y verdes según tendencia y fortaleza.
        - activos_alcistas_operadores: devuelve un listado de los activos que están por encima de la MM de 10 semanas (Operadores).
        - activos_alcistas_inversores: devuelve un listado de los activos que están por debajo de la MM de 30 semanas (Inversores).
    """
    
    import pandas as pd
    import pandas_ta as ta
    import yfinance as yf
    
    data = getDataYfMulti(activos, tipo = "period", interval = "1wk", period = "5y", swap = False)["Close"]
    data = pd.DataFrame(data)
    data.dropna(inplace = True)

    # Defino datos de estrategia
    MyStrategy = ta.Strategy(
    name = "stanWeinstein",
    ta = [
        {"kind" : "ema", "close" : "close", "length" : 10},
        {"kind" : "ema", "close" : "close", "length" : 30},
    ]
    )
    
    # Creamos DFs para operadores:
    df_distancia_10 = pd.DataFrame(index = data.index)  # Es el feature numerico (P/EMA10 - 1) (Variación no porcentual)
    df_fortaleza_10 = pd.DataFrame(index = data.index)  # Es el feature categorico de fortaleza de tend: si últimos 3 valores de df_distancia son crecientes, validamos tendencia creciente.            
    
    # Creamos DFs para inversores:
    df_distancia_30 = pd.DataFrame(index = data.index)  # Es el feature numerico (P/EMA30 - 1) (Variación no porcentual)
    df_fortaleza_30 = pd.DataFrame(index = data.index)  # Es el feature categorico de fortaleza de tend: si últimos 3 valores de df_distancia son crecientes, validamos tendencia creciente. 
    
    #tendencia = pd.DataFrame()
    #fortaleza = pd.DataFrame()

    #df_resumen = pd.DataFrame() # ==> Resumen de todos los activos listados en una sola tabla.
    dict_resumenPorActivo = {}  # ==> Resumen por activo (un diccionario de diccionarios).

    i = 0
    
    for activo in data.columns:
        try:
            datat = pd.DataFrame(data[activo])
            datat.rename(columns = {activo : "Close"}, inplace = True)   # Tengo que renombrar el ticker por "Close", sino no se calcula la MACD.
            datat.ta.strategy(MyStrategy)
            datat.dropna(inplace = True)
            datat["distancia_10"] = ((datat["Close"] / datat["EMA_10"]) - 1) * 100
            datat["distancia_30"] = ((datat["Close"] / datat["EMA_30"]) - 1) * 100
            
            # EMA 10 week: OPERADORES
            # Si la última posición es alcista:
            if datat["distancia_10"][-1] > 0:  
                datat["fortaleza_10"] = np.where(((datat["distancia_10"] > datat["distancia_10"].shift(1)) & 
                                              (datat["distancia_10"].shift(1) > datat["distancia_10"].shift(2))), 
                                            "Creciente",
                                    np.where(((datat["distancia_10"] < datat["distancia_10"].shift(1)) 
                                              & (datat["distancia_10"].shift(1) < datat["distancia_10"].shift(2))),
                                            "Decreciente", "No determinada"))
            # Si la última posición es bajista:
            if datat["distancia_10"][-1] < 0:  
                datat["fortaleza_10"] = np.where(((datat["distancia_10"] < datat["distancia_10"].shift(1)) & 
                                              (datat["distancia_10"].shift(1) < datat["distancia_10"].shift(2))), 
                                            "Creciente",
                                    np.where(((datat["distancia_10"] > datat["distancia_10"].shift(1)) 
                                              & (datat["distancia_10"].shift(1) > datat["distancia_10"].shift(2))),
                                            "Decreciente", "No determinada"))
            
            # EMA 30 week: INVERSORES
            # Si la última posición es alcista:
            if datat["distancia_30"][-1] > 0:  
                datat["fortaleza_30"] = np.where(((datat["distancia_30"] > datat["distancia_30"].shift(1)) & 
                                              (datat["distancia_30"].shift(1) > datat["distancia_30"].shift(2))), 
                                            "Creciente",
                                    np.where(((datat["distancia_30"] < datat["distancia_30"].shift(1)) 
                                              & (datat["distancia_30"].shift(1) < datat["distancia_30"].shift(2))),
                                            "Decreciente", "No determinada"))
            # Si la última posición es bajista:
            if datat["distancia_30"][-1] < 0:  
                datat["fortaleza_30"] = np.where(((datat["distancia_30"] < datat["distancia_30"].shift(1)) & 
                                              (datat["distancia_30"].shift(1) < datat["distancia_30"].shift(2))), 
                                            "Creciente",
                                    np.where(((datat["distancia_30"] > datat["distancia_30"].shift(1)) 
                                              & (datat["distancia_30"].shift(1) > datat["distancia_30"].shift(2))),
                                            "Decreciente", "No determinada"))
            
            
            # Cargamos distancias y fortalezas en tablas:
            df_distancia_10[activo] = datat["distancia_10"] 
            df_fortaleza_10[activo] = datat["fortaleza_10"]
            df_distancia_10 = df_distancia_10.dropna()
            df_fortaleza_10 = df_fortaleza_10.dropna()
            
            df_distancia_30[activo] = datat["distancia_30"] 
            df_fortaleza_30[activo] = datat["fortaleza_30"]
            df_distancia_30 = df_distancia_30.dropna()
            df_fortaleza_30 = df_fortaleza_30.dropna()

            
            # Tomamos los datos de las tablas:
            dato_df_distancia_10 = "Alcista" if df_distancia_10[activo][-1] > 0 else "Bajista"
            dato_df_fortaleza_10 = df_fortaleza_10[activo][-1]
            dato_df_distancia_30 = "Alcista" if df_distancia_30[activo][-1] > 0 else "Bajista"
            dato_df_fortaleza_30 = df_fortaleza_30[activo][-1]
             
            # Cargamos el activo:
            dict_resumenPorActivo[activo] = {"tendencia_operadores(10w)" : dato_df_distancia_10,
                                             "fortaleza_operadores(10w)" : dato_df_fortaleza_10,
                                             "distancia_P-MM10w" : round(df_distancia_10[activo][-1],2),
                                             "tendencia_inversores(30w)" : dato_df_distancia_30,
                                             "fortaleza_inversores(30w)" : dato_df_fortaleza_30,
                                             "distancia_P-MM30w" : round(df_distancia_30[activo][-1],2)}
            
            '''
            if df_distancia_10[activo][-1] >= 0:
                if df_fortaleza_10[activo][-1] == "Creciente":
                    dict_resumenPorActivo[activo] = {"tendencia" : "Alcista", "fortaleza" : "Creciente"}
                elif df_fortaleza[activo][-1] == "Decreciente":
                    dict_resumenPorActivo[activo] = {"tendencia" : "Alcista", "fortaleza" : "Decreciente"}
            
            elif df_distancia[activo][-1] < 0:
                if df_fortaleza[activo][-1] == "Creciente":
                    dict_resumenPorActivo[activo] = {"tendencia" : "Bajista", "fortaleza" : "Creciente"}
                elif df_fortaleza[activo][-1] == "Decreciente":
                    dict_resumenPorActivo[activo] = {"tendencia" : "Bajista", "fortaleza" : "Decreciente"}
            '''
      
            
            

            tabla_final_activos_en_columnas = pd.DataFrame(dict_resumenPorActivo)
            tabla_final_activos_en_filas = tabla_final_activos_en_columnas.transpose()
            i += 1
            print(f"[{activo}]Procesando {i} de {len(data.columns)} activos ...")

        except:
            pass
    
    tabla = tabla_final_activos_en_filas
    
    def set_styles(tabla):
        def highlight_tend(val, color_if_true, color_if_false):
            color = color_if_true if val == "Alcista" else color_if_false
            return f"background-color: {color}"
        
        def highlight_notDet(val, color_if_true):
            color = color_if_true if val == "No determinada" else None
            return f"background-color: {color}"
        

        highlighted_rows = np.where((tabla['tendencia_operadores(10w)'] == "Alcista") & (tabla['fortaleza_operadores(10w)'] == "Creciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['tendencia_operadores(10w)'] == "Alcista") & (tabla['fortaleza_operadores(10w)'] == "Decreciente"), 
                                    'background-color: #FA6B84', 
                                    np.where((tabla['tendencia_operadores(10w)'] == "Bajista") & (tabla['fortaleza_operadores(10w)'] == "Decreciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['tendencia_operadores(10w)'] == "Bajista") & (tabla['fortaleza_operadores(10w)'] == "Creciente"), 
                                    'background-color: #FA6B84', 
                                            
                                    np.where((tabla['tendencia_inversores(30w)'] == "Alcista") & (tabla['fortaleza_inversores(30w)'] == "Creciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['tendencia_inversores(30w)'] == "Alcista") & (tabla['fortaleza_inversores(30w)'] == "Decreciente"), 
                                    'background-color: #FA6B84', 
                                    np.where((tabla['tendencia_inversores(30w)'] == "Bajista") & (tabla['fortaleza_inversores(30w)'] == "Decreciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['tendencia_inversores(30w)'] == "Bajista") & (tabla['fortaleza_inversores(30w)'] == "Creciente"), 
                                    'background-color: #FA6B84', ''))))))))
        

        styler = tabla.style.apply(lambda _: highlighted_rows, subset=["fortaleza_operadores(10w)", "fortaleza_inversores(30w)"]) \
                            .applymap(highlight_tend, color_if_true='#90EF90', color_if_false='#FA6B84', 
                              subset=['tendencia_operadores(10w)', 'tendencia_inversores(30w)']) \
                            .applymap(lambda _: "color: black; font-weight: bold") \
                            .applymap(highlight_notDet, color_if_true = "grey") \
                            .applymap(lambda _: "color: white; font-weight: bold", subset=["distancia_P-MM10w","distancia_P-MM30w"]) \
                            .format('{:.2f}%', subset=["distancia_P-MM10w","distancia_P-MM30w"])
        

        return styler
    
    estilado = set_styles(tabla)
    
    # Generación de reporte en excel (opcional)
    if report == True:
        from datetime import date
        today = date.today()
        estilado2 = estilado.applymap(lambda _: "color: grey; font-weight: bold", subset = ["distancia_P-MM10w", "distancia_P-MM30w"])
        estilado2.to_excel(f'reporteStanW-{today}.xlsx')
    
    
    
    activos_alcistas_operadores = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["tendencia_operadores(10w)"] == "Alcista"].index)
    activos_alcistas_inversores = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["tendencia_inversores(30w)"] == "Alcista"].index)
    
    
    
    return (tabla_final_activos_en_filas, 
            tabla_final_activos_en_columnas, 
            estilado, 
            activos_alcistas_operadores, 
            activos_alcistas_inversores)



def getStanSignals(activos, tipo):
    
    """
    Esta función devuelve los cruces vigentes del precio respecto de las medias exponenciales de 10 y 30 semanas del listado de activos
    solicitado, así como el sentido del cruce, la fecha de la señal, el precio de la señal, el rendimiento acumulado a la fecha
    y el rendimiento acumulado, todo ordenado en forma descendente por cantidad de días transcurridos desde la señal.
    
    # Inputs:
        - activos: es una lista de activos a analizar. Ej.: adrs = ["GGAL", "BBAR", "LOMA", "CEPU"]. También hay sets de activos preconfigurados en la función:
            - "lideres": si se desea cargar todo el panel lider de Argentina;
            - "general": si se desea cargar todo el panel general de Argentina;
            - "adrs": si se desea cargar todo el listado de ADRs argentinos (en usd);
            - "sectors": si se desea cargar el listado de sectores de la economía (ETFs representativos de USA);
            - "cedears": si se desea cargar todo el listado de certificados de depósito de activos del exterior que cotizan en Argentina (puede demorar bastante).
        - tipo: determina la referencia: "inversores" para la MM de 30 semanas u "operadores" para la MM de 10 semanas.
    
    # Outputs:
        - stan_cross_signals: diccionario con las señales vigentes;
        - stan_cross_signals_df: tabla con las señales vigentes;
        - stan_cross_signals_df_formateado: tabla con las señales vigentes formateada con escala de colores según rendimiento
        acumulado anualizado desde la señal.
        
    """
    
    import pandas as pd
    import pandas_ta as ta
    import yfinance as yf
    
    interval = "1wk"
    period = "10y"

    
    data = getDataYfMulti(activos, tipo = "period", interval = interval, period = period, swap = False)["Close"]
    data = pd.DataFrame(data)
    data.dropna(inplace = True)

    if tipo == "operadores":
        mm_length = 10
    elif tipo == "inversores":
        mm_length = 30
    else:
        return "Indicar tipo = 'operadores' o 'inversores'"
    
    # Defino datos de estrategia
    MyStrategy = ta.Strategy(
    name = "stanW",
    ta = [
        {"kind" : "ema", "close" : "close", "length" : mm_length},
    ]
    )

    df_distancia = pd.DataFrame(index = data.index)  # Es el feature numerico (P/EMA10o30 - 1) (Variación no porcentual)
    df_fortaleza = pd.DataFrame(index = data.index)  # Es el feature categorico de fortaleza de tend: si últimos 3 valores de df_distancia son crecientes, validamos tendencia creciente.            

    stan_cross_signals = {}

    for activo in data.columns:
        try:
            # Iteramos por cada activo
            # Armamos el dataframe OHLC y calculamos componentes del MACD
            datat = pd.DataFrame(data[activo])
            datat.rename(columns = {activo : "Close"}, inplace = True)   # Tengo que renombrar el ticker por "Close", sino no se calcula la MACD.
            datat.ta.strategy(MyStrategy)
            datat.dropna(inplace = True)
            datat["distancia"] = (datat["Close"] / datat[f"EMA_{mm_length}"]) - 1

            # Obtenemos el precio actual
            actual_price = round(datat["Close"][-1],2)
            # Determinamos posición y cambios de las EMA de 50 y 200:
            datat["posicionEMAs"] = np.where(datat["distancia"] > 0, 1, -1)
            datat["posicionEMAs_change"] = np.where(datat["posicionEMAs"] > datat["posicionEMAs"].shift(1), "Compra", 
                                                        np.where(datat["posicionEMAs"] < datat["posicionEMAs"].shift(1), "Venta", ""))


            # 1) Caso de Long, calculamos estadísticas:
            if datat["posicionEMAs"][-1] > 0:
                last_buy_signal_breakout = datat[datat["posicionEMAs_change"] == "Compra"].iloc[-1]
                price_lbs_breakout = round(last_buy_signal_breakout["Close"],2)
                date_lbs_breakout = datetime(last_buy_signal_breakout.name.year, 
                                                    last_buy_signal_breakout.name.month, 
                                                    last_buy_signal_breakout.name.day)
                days_since_bs_breakout = (datetime.today() - date_lbs_breakout).days
                rend_since_bs_breakout = round(((actual_price / price_lbs_breakout) - 1) * 100, 2)
                rend_since_bs_annualized_breakout = round((((1 + ((actual_price / price_lbs_breakout) - 1)) ** (365/days_since_bs_breakout)) - 1) * 100,2)
                stan_cross_signals[activo] = {"type" : "LONG", 
                                                        "precioSeñal" : price_lbs_breakout, 
                                                        "fechaSeñal": date_lbs_breakout.strftime("%Y-%m-%d"), 
                                                        "diasDesdeSeñal" : days_since_bs_breakout,
                                                        "precioActual" : round(actual_price,2),
                                                        "rendAcumDesdeSeñal":  rend_since_bs_breakout, 
                                                        "rendAnualDesdeSeñal" : rend_since_bs_annualized_breakout}


            # 2) Caso de Short, calculamos estadísticas:     
            elif datat["posicionEMAs"][-1] < 0:
                last_sell_signal_breakout = datat[datat["posicionEMAs_change"] == "Venta"].iloc[-1]
                price_lss_breakout = round(last_sell_signal_breakout["Close"],2)
                date_lss_breakout = datetime(last_sell_signal_breakout.name.year, 
                                                last_sell_signal_breakout.name.month, 
                                                last_sell_signal_breakout.name.day)
                days_since_ss_breakout = (datetime.today() - date_lss_breakout).days
                rend_since_ss_breakout = round(((price_lss_breakout / actual_price) - 1) * 100, 2)
                rend_since_ss_annualized_breakout = round((((1 + ((price_lss_breakout / actual_price) - 1)) ** (365/days_since_ss_breakout)) - 1) * 100,2)
                stan_cross_signals[activo] = {"type" : "SHORT", 
                                                "precioSeñal" : price_lss_breakout, 
                                                "fechaSeñal": date_lss_breakout.strftime("%Y-%m-%d"), 
                                                "diasDesdeSeñal" : days_since_ss_breakout,
                                                "precioActual" : round(actual_price,2),
                                                "rendAcumDesdeSeñal":  rend_since_ss_breakout, 
                                                "rendAnualDesdeSeñal" : rend_since_ss_annualized_breakout}
            print(f"{activo} - procesado!")
        except:
            print(f"{activo} - no pudo ser procesado")
            pass

    stan_cross_signals_df = pd.DataFrame(stan_cross_signals).transpose().sort_values(by = "diasDesdeSeñal", ascending = True)
    stan_cross_signals_df_formateado = stan_cross_signals_df.style.background_gradient(axis=0, gmap=stan_cross_signals_df['rendAnualDesdeSeñal'], 
                                                                                         cmap='RdYlGn') \
    .format('{:.2f}%', subset=["rendAcumDesdeSeñal", "rendAnualDesdeSeñal"]) \
    .format('{:.2f}', subset=["precioSeñal", "precioActual"])
    
    
    return stan_cross_signals, stan_cross_signals_df, stan_cross_signals_df_formateado

In [3]:
tabla, _ , tabla_estilada, alcistas_operadores, alcistas_inversores = filterStanW("adrs", report = False)

[BBAR]Procesando 1 de 15 activos ...
[BMA]Procesando 2 de 15 activos ...
[CEPU]Procesando 3 de 15 activos ...
[CRESY]Procesando 4 de 15 activos ...
[EDN]Procesando 5 de 15 activos ...
[GGAL]Procesando 6 de 15 activos ...
[IRS]Procesando 7 de 15 activos ...
[LOMA]Procesando 8 de 15 activos ...
[PAM]Procesando 9 de 15 activos ...
[SUPV]Procesando 10 de 15 activos ...
[TEO]Procesando 11 de 15 activos ...
[TGS]Procesando 12 de 15 activos ...
[TS]Procesando 13 de 15 activos ...
[TX]Procesando 14 de 15 activos ...
[YPF]Procesando 15 de 15 activos ...


In [4]:
# filterStanW - output - tabla
tabla

,tendencia_operadores(10w),fortaleza_operadores(10w),distancia_P-MM10w,tendencia_inversores(30w),fortaleza_inversores(30w),distancia_P-MM30w
BBAR,Alcista,Creciente,7.89,Alcista,Creciente,13.84
BMA,Alcista,Creciente,13.1,Alcista,Creciente,28.62
CEPU,Alcista,No determinada,11.42,Alcista,No determinada,32.2
CRESY,Alcista,No determinada,4.4,Alcista,No determinada,16.19
EDN,Alcista,Creciente,12.18,Alcista,Creciente,33.89
GGAL,Alcista,Creciente,9.84,Alcista,Creciente,20.59
IRS,Alcista,Creciente,5.28,Alcista,Creciente,12.75
LOMA,Alcista,Creciente,7.78,Alcista,Creciente,13.01
PAM,Alcista,Creciente,2.92,Alcista,No determinada,11.65
SUPV,Alcista,No determinada,14.91,Alcista,Creciente,35.7


In [6]:
# filterStanW - output - tabla con estilos
tabla_estilada

,tendencia_operadores(10w),fortaleza_operadores(10w),distancia_P-MM10w,tendencia_inversores(30w),fortaleza_inversores(30w),distancia_P-MM30w
BBAR,Alcista,Creciente,7.89%,Alcista,Creciente,13.84%
BMA,Alcista,Creciente,13.10%,Alcista,Creciente,28.62%
CEPU,Alcista,No determinada,11.42%,Alcista,No determinada,32.20%
CRESY,Alcista,No determinada,4.40%,Alcista,No determinada,16.19%
EDN,Alcista,Creciente,12.18%,Alcista,Creciente,33.89%
GGAL,Alcista,Creciente,9.84%,Alcista,Creciente,20.59%
IRS,Alcista,Creciente,5.28%,Alcista,Creciente,12.75%
LOMA,Alcista,Creciente,7.78%,Alcista,Creciente,13.01%
PAM,Alcista,Creciente,2.92%,Alcista,No determinada,11.65%
SUPV,Alcista,No determinada,14.91%,Alcista,Creciente,35.70%


In [7]:
# filterStanW - output - Lista con activos alcistas perspectiva operadores
alcistas_operadores

['BBAR',
 'BMA',
 'CEPU',
 'CRESY',
 'EDN',
 'GGAL',
 'IRS',
 'LOMA',
 'PAM',
 'SUPV',
 'TEO',
 'TGS']

In [8]:
# filterStanW - output - Lista con activos alcistas perspectiva inversores
alcistas_inversores

['BBAR',
 'BMA',
 'CEPU',
 'CRESY',
 'EDN',
 'GGAL',
 'IRS',
 'LOMA',
 'PAM',
 'SUPV',
 'TEO',
 'TGS',
 'YPF']

In [9]:
dic, tabla, tabla_estilada = getStanSignals("adrs", tipo = "inversores")

BBAR - procesado!
BMA - procesado!
CEPU - procesado!
CRESY - procesado!
EDN - procesado!
GGAL - procesado!
IRS - procesado!
LOMA - procesado!
PAM - procesado!
SUPV - procesado!
TEO - procesado!
TGS - procesado!
TS - procesado!
TX - procesado!
YPF - procesado!


In [10]:
# getStanSignals - output - diccionario
dic

{'BBAR': {'type': 'LONG',
  'precioSeñal': 5.24,
  'fechaSeñal': '2023-11-20',
  'diasDesdeSeñal': 64,
  'precioActual': 5.58,
  'rendAcumDesdeSeñal': 6.49,
  'rendAnualDesdeSeñal': 43.12},
 'BMA': {'type': 'LONG',
  'precioSeñal': 19.79,
  'fechaSeñal': '2023-11-13',
  'diasDesdeSeñal': 71,
  'precioActual': 30.22,
  'rendAcumDesdeSeñal': 52.7,
  'rendAnualDesdeSeñal': 781.33},
 'CEPU': {'type': 'LONG',
  'precioSeñal': 7.46,
  'fechaSeñal': '2023-11-20',
  'diasDesdeSeñal': 64,
  'precioActual': 9.29,
  'rendAcumDesdeSeñal': 24.53,
  'rendAnualDesdeSeñal': 249.44},
 'CRESY': {'type': 'LONG',
  'precioSeñal': 7.27,
  'fechaSeñal': '2023-10-09',
  'diasDesdeSeñal': 106,
  'precioActual': 9.28,
  'rendAcumDesdeSeñal': 27.65,
  'rendAnualDesdeSeñal': 131.76},
 'EDN': {'type': 'LONG',
  'precioSeñal': 16.4,
  'fechaSeñal': '2023-11-20',
  'diasDesdeSeñal': 64,
  'precioActual': 20.29,
  'rendAcumDesdeSeñal': 23.72,
  'rendAnualDesdeSeñal': 236.66},
 'GGAL': {'type': 'LONG',
  'precioSeñal

In [11]:
# getStanSignals - output - tabla
tabla

,type,precioSeñal,fechaSeñal,diasDesdeSeñal,precioActual,rendAcumDesdeSeñal,rendAnualDesdeSeñal
TX,SHORT,38.86,2024-01-15,8,38.65,0.54,28.05
TS,SHORT,32.31,2024-01-08,15,31.98,1.03,28.38
BBAR,LONG,5.24,2023-11-20,64,5.58,6.49,43.12
CEPU,LONG,7.46,2023-11-20,64,9.29,24.53,249.44
EDN,LONG,16.4,2023-11-20,64,20.29,23.72,236.66
GGAL,LONG,15.32,2023-11-20,64,18.18,18.67,165.43
IRS,LONG,8.12,2023-11-20,64,8.69,7.02,47.24
LOMA,LONG,6.5,2023-11-20,64,7.32,12.62,96.91
PAM,LONG,45.59,2023-11-20,64,48.13,5.57,36.23
SUPV,LONG,3.03,2023-11-20,64,4.18,37.95,526.5


In [12]:
# getStanSignals - output - tabla con estilos
tabla_estilada

,type,precioSeñal,fechaSeñal,diasDesdeSeñal,precioActual,rendAcumDesdeSeñal,rendAnualDesdeSeñal
TX,SHORT,38.86,2024-01-15,8,38.65,0.54%,28.05%
TS,SHORT,32.31,2024-01-08,15,31.98,1.03%,28.38%
BBAR,LONG,5.24,2023-11-20,64,5.58,6.49%,43.12%
CEPU,LONG,7.46,2023-11-20,64,9.29,24.53%,249.44%
EDN,LONG,16.40,2023-11-20,64,20.29,23.72%,236.66%
GGAL,LONG,15.32,2023-11-20,64,18.18,18.67%,165.43%
IRS,LONG,8.12,2023-11-20,64,8.69,7.02%,47.24%
LOMA,LONG,6.50,2023-11-20,64,7.32,12.62%,96.91%
PAM,LONG,45.59,2023-11-20,64,48.13,5.57%,36.23%
SUPV,LONG,3.03,2023-11-20,64,4.18,37.95%,526.50%
